# **Modelling and Tuning**

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, LassoCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold


In [2]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# Ignore ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [3]:
import pickle

# Load train/val/test split data from notebook3
with open(r'Nata_Files\\train_test_split.pkl', 'rb') as f:
    notebook3_data = pickle.load(f)

# Core datasets
X = notebook3_data.get('X')
y = notebook3_data.get('y')

# Splits and processed feature sets
X_train = notebook3_data.get('X_train')
y_train = notebook3_data.get('y_train')
X_val = notebook3_data.get('X_val')
y_val = notebook3_data.get('y_val')
X_test = notebook3_data.get('X_test')
y_test = notebook3_data.get('y_test')
X_train_val = notebook3_data.get('X_train_val')
y_train_val = notebook3_data.get('y_train_val')

numeric_cols = notebook3_data.get('numeric_cols')
kf = notebook3_data.get('kf')
rkf = notebook3_data.get('rkf')
skf = notebook3_data.get('skf')

print("Train/Val/Test split data loaded successfully!")
if X is not None and y is not None:
    print(f"Full dataset X shape: {X.shape} | y shape: {y.shape}")
if X_train is not None:
    print(f"X_train shape: {X_train.shape}")
if X_val is not None:
    print(f"X_val shape: {X_val.shape}")
if X_test is not None:
    print(f"X_test shape: {X_test.shape}")
if kf is not None:
    try:
        print(f"kf splits: {kf.get_n_splits()}")
    except Exception:
        print("kf loaded (object), get_n_splits() unavailable for this object")
if rkf is not None:
    try:
        print(f"rkf splits: {rkf.get_n_splits()}")
    except Exception:
        print("rkf loaded (object), get_n_splits() unavailable for this object")

Train/Val/Test split data loaded successfully!
Full dataset X shape: (5196, 14) | y shape: (5196,)
X_train shape: (3117, 14)
X_val shape: (1039, 14)
X_test shape: (1040, 14)
kf splits: 10
rkf splits: 14


In [4]:
class Predictor:
    def __init__(self, model):
        self.model = model

    def fit(self, X, y):
        self.model.fit(X, y)

    def predict_proba(self, X_val):
        return self.model.predict_proba(X_val)
    
    def predict(self, X_val):
        return self.model.predict(X_val)

## **Model Selection**

In [5]:
logr = Predictor(LogisticRegression())
logr.fit(X_train, y_train)
logr_proba = logr.predict_proba(X_val)[:,1]
logr_pred = logr.predict(X_val)
logr_proba_tr = logr.predict_proba(X_train)[:,1]
logr_pred_tr = logr.predict(X_train)

dtc = Predictor(DecisionTreeClassifier(max_depth= 5))
dtc.fit(X_train, y_train)
dtc_proba = dtc.predict_proba(X_val)[:,1]
dtc_pred = dtc.predict(X_val)
dtc_proba_tr = logr.predict_proba(X_train)[:,1]
dtc_pred_tr = logr.predict(X_train)


rf = Predictor(RandomForestClassifier())
rf.fit(X_train, y_train)
rf_proba = rf.predict_proba(X_val)[:,1]
rf_pred = rf.predict(X_val)
rf_proba_tr = logr.predict_proba(X_train)[:,1]
rf_pred_tr = logr.predict(X_train)

knn_clf = Predictor(KNeighborsClassifier())
knn_clf.fit(X_train, y_train)
knn_clf_proba = knn_clf.predict_proba(X_val)[:,1]
knn_clf_pred = knn_clf.predict(X_val)
knn_clf_proba_tr = logr.predict_proba(X_train)[:,1]
knn_clf_pred_tr = logr.predict(X_train)


In [6]:
def get_metrics(y_val, y_proba, y_pred, model, dataset):
    return {
        "Model" : model,
        "Set" : dataset,
        "AUC": roc_auc_score(y_val, y_proba),
        "Accuracy": accuracy_score(y_val, y_pred),
    }

In [7]:
models_metrics = []

models_metrics.append(get_metrics(y_train, logr_proba_tr, logr_pred_tr, "Logistic Regression", "Train"))
models_metrics.append(get_metrics(y_train, dtc_proba_tr, dtc_pred_tr, "DTClassifier", "Train"))
models_metrics.append(get_metrics(y_train, rf_proba_tr, rf_pred_tr, "Random Forest", "Train"))
models_metrics.append(get_metrics(y_train, knn_clf_proba_tr, knn_clf_pred_tr, "KNClassifier", "Train"))

models_metrics.append(get_metrics(y_val, logr_proba, logr_pred, "Logistic Regression", "Validation"))
models_metrics.append(get_metrics(y_val, dtc_proba, dtc_pred, "DTClassifier", "Validation"))
models_metrics.append(get_metrics(y_val, rf_proba, rf_pred, "Random Forest", "Validation"))
models_metrics.append(get_metrics(y_val, knn_clf_proba, knn_clf_pred, "KNClassifier", "Validation"))

In [8]:
df_models_metrics = pd.DataFrame(models_metrics)
df_models_metrics = df_models_metrics.pivot_table(
    index=["Model", "Set"],
    values=["AUC", "Accuracy"]
)

df_models_metrics

AUC  Accuracy
Model               Set                           
DTClassifier        Train       0.797320  0.741739
                    Validation  0.768210  0.728585
KNClassifier        Train       0.797320  0.741739
                    Validation  0.774752  0.726660
Logistic Regression Train       0.797320  0.741739
                    Validation  0.809067  0.750722
Random Forest       Train       0.797320  0.741739
                    Validation  0.838315  0.769009

### **Feature Selection**

In [14]:
lasso = LassoCV()
lasso.fit(X_train, y_train)

,eps,0.001
,n_alphas,'deprecated'
,alphas,'warn'
,fit_intercept,True
,precompute,'auto'
,max_iter,1000
,tol,0.0001
,copy_X,True
,cv,None
,verbose,False
,n_jobs,None


In [15]:
coef = pd.Series(lasso.coef_, index = X_train.columns)
coef

ambient_humidity     0.000000
baking_duration     -0.135690
cooling_period      -0.017361
cream_fat_content   -0.000000
egg_temperature      0.044335
egg_yolk_count       0.183838
final_temperature   -0.061456
lemon_zest_ph        0.001352
oven_temperature    -0.000000
preheating_time     -0.000000
salt_ratio          -0.003262
sugar_content        0.049816
vanilla_extract      0.057433
is_lisboa           -0.000000
dtype: float64

In [16]:
coef[coef==0]


ambient_humidity     0.0
cream_fat_content   -0.0
oven_temperature    -0.0
preheating_time     -0.0
is_lisboa           -0.0
dtype: float64

In [17]:
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")


Lasso picked 9 variables and eliminated the other 5 variables


In [18]:
coef.sort_values(ascending=False)


egg_yolk_count       0.183838
vanilla_extract      0.057433
sugar_content        0.049816
egg_temperature      0.044335
lemon_zest_ph        0.001352
cream_fat_content   -0.000000
is_lisboa           -0.000000
preheating_time     -0.000000
ambient_humidity     0.000000
oven_temperature    -0.000000
salt_ratio          -0.003262
cooling_period      -0.017361
final_temperature   -0.061456
baking_duration     -0.135690
dtype: float64

In [20]:
selected_features = X_train.columns[lasso.coef_ != 0]

X_train_lr = X_train[selected_features]
X_val_lr = X_val[selected_features]

In [25]:
logr = LogisticRegression()
logr.fit(X_train_lr, y_train)

logr_proba = logr.predict_proba(X_val_lr)[:,1]
logr_pred = logr.predict(X_val_lr)

print(f"AUC: {roc_auc_score(y_val, logr_proba)}, Accuracy: {accuracy_score(y_val, logr_pred)}")

AUC: 0.8087830814743744, Accuracy: 0.7526467757459095
